In [ ]:
#new models

In [1]:
pip install langchain


   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.3/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 465.6 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 465.6 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 465.6 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 465.6 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 465.6 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 465.6 kB/s eta 0:00:02
   -------------------- ------------------- 0.5/1.0 MB 465.6 kB/s eta 0:00:02
   ------------------------------- -------- 0.8/1.0 MB 248.6 kB/s eta 0:00:01
   ---------------------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)

response = llm.invoke("create quiz on ml linear regression topic with mcqs and option")
print(response.content)

Here's a quiz on ML Linear Regression with MCQs and options:

**Quiz: ML Linear Regression**

**1. What is the primary goal of Linear Regression in Machine Learning?**

A) To classify data into different categories
B) To predict a continuous output variable based on one or more input features
C) To reduce the dimensionality of data
D) To identify the most important features in a dataset

**Answer: B) To predict a continuous output variable based on one or more input features**

**2. Which of the following is a type of Linear Regression?**

A) Logistic Regression
B) Decision Trees
C) Simple Linear Regression
D) K-Means Clustering

**Answer: C) Simple Linear Regression**

**3. What is the equation for Simple Linear Regression?**

A) y = mx + b
B) y = mx - b
C) y = mx / b
D) y = mx * b

**Answer: A) y = mx + b**

**4. What is the role of the coefficient 'm' in the Simple Linear Regression equation?**

A) It represents the y-intercept
B) It represents the slope of the regression line
C) It

In [7]:
pip install chromadb

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mysql-connector-python 8.0.33 requires protobuf<=3.20.3,>=3.11.0, but you have protobuf 5.29.0 which is incompatible.
streamlit 1.34.0 requires protobuf<5,>=3.20, but you have protobuf 5.29.0 which is incompatible.
tensorflow-intel 2.16.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/617.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/617.9 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/617.9 kB ? eta -:--:--
   ---------------- ----------------------- 262.1/617.9 kB ? eta -:--:--
   -------------------------------- ----- 524.3/617.9 kB 493.7 kB/s eta 0:00:01
   -------------------------------- ----- 524.3/617.9 kB 493.7 kB/s eta 0:00:01
   -------------------------------------- 617.9/617.9 kB 465.0 kB/s eta 0:00:00
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   